In [1]:
import gdown

url = 'https://drive.google.com/u/0/uc?id=16gqSgqCiIQAFlZNFV_tdGQK5Q1mijc7u'
output = 'celeba.zip'
#gdown.download(url, output, quiet=False)

In [2]:
#!unzip celeba.zip

In [3]:
#cd celeba/

In [4]:
#cd celeba/

In [5]:
#!rm -R img_align_celeba/

In [6]:
#!unzip img_align_celeba.zip

In [7]:
#cd ..

In [8]:
#cd ..

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

from tqdm import tqdm_notebook as tqdm

In [11]:
BS = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

celeba_transforms = transforms.Compose([
    transforms.CenterCrop((148, 148)),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
    


my_dataset = torchvision.datasets.CelebA('./celeba/',
                                           transform=celeba_transforms,
                                           download=False)
all_size = len(my_dataset)
print('all train+val samples:', all_size)

all train+val samples: 162770


In [12]:
val_size = 2000
train_size = all_size - val_size
train_set, val_set = torch.utils.data.random_split(my_dataset, [train_size, val_size])

In [13]:
train_loader = DataLoader(train_set, batch_size=BS, shuffle=True,
                          num_workers=2, pin_memory=True, drop_last=True)

val_loader = DataLoader(val_set, batch_size=1, shuffle=False)
fid_loader = DataLoader(val_set, batch_size=BS, shuffle=False, drop_last=True)

В этой домашней работе вам предлагается повторить результаты статьи VAE+NF (https://arxiv.org/pdf/1611.05209.pdf).

Основная часть домашнего задания - чтение статьи и повторение результатов, поэтому обязательно прочитайте не только ее, но и другие основные статьи про потоки того времени:

1. https://arxiv.org/abs/1505.05770
2. https://arxiv.org/abs/1605.08803
3. https://arxiv.org/abs/1705.07057
4. http://arxiv.org/abs/1807.03039




### Задача 1 (0.1 балла, но если не сделаете, за всю домашку ноль):

Для начала предлагаю попробовать обучить обычный VAE на Celeba до нормального качества, померить FID и запомнить для будущего сравнения


### CONFIG

In [14]:
from my_utils import set_seed, count_parameters

set_seed(21)

In [15]:
class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

config = {
    'image_size': 64,
    'batch_size': BS,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'num_epochs': 10,
    'grad_clip_value': 5
}
config = DotDict(config)

### NF config
config['hid_size'] = 24

### Training and Optimization config
config['lr_start'] = 1e-3

### WandB

In [16]:
! pip install wandb

In [17]:
import wandb
wandb.login(key='6aa2251ef1ea5e572e6a7608c0152db29bd9a294')

def wandb_start(config, run_name):
    wandb.init(project="dgm-ht3", config=config)
    wandb.run.name = run_name


wandb: Currently logged in as: kirili4ik (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Data

In [18]:
'''plt.figure(figsize=(30, 30))

for index, image_attr in enumerate(fid_loader):
    
    image = image_attr[0].to(config.device)
        
    print(int(image.min()), int(image.max()), image.size())
    image = image[0]
    if index >= 10: break
    plt.subplot(10, 1, index+1)
    plt.imshow((image.squeeze().cpu().permute(1, 2, 0) + 1) / 2)
    plt.axis('off')

plt.show()'''

"plt.figure(figsize=(30, 30))\n\nfor index, image_attr in enumerate(fid_loader):\n    \n    image = image_attr[0].to(config.device)\n        \n    print(int(image.min()), int(image.max()), image.size())\n    image = image[0]\n    if index >= 10: break\n    plt.subplot(10, 1, index+1)\n    plt.imshow((image.squeeze().cpu().permute(1, 2, 0) + 1) / 2)\n    plt.axis('off')\n\nplt.show()"

### FID

In [19]:
from inception import InceptionV3

classifier = InceptionV3()
classifier.to(config.device)
print()

from my_calculate_fid_VAE import calculate_fid as fid_VAE
from my_calculate_fid_NF  import calculate_fid as fid_NF

### VAE Model 

In [ ]:
config = {
    'image_size': 64,
    'batch_size': 128,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'num_epochs': 30,
    'grad_clip_value': 1.5
}
config = DotDict(config)

### VAE config
config['z_size'] = 128


### Training and Optimization config
config['lr_start'] = 0.005

wandb_start(config, 'VAE-...')

In [ ]:
from VAE import VAE

model = VAE(
            z_size=config.z_size,
            im_size=config.image_size,
            device=config.device
        ).to(config.device)

wandb.watch(model)

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr_start)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.95, verbose=True)

In [ ]:
import torch.nn.functional as F

def loss_function(recon_x, x, mu, logvar):
    batch_size = recon_x.shape[0]
    MSE = F.mse_loss(recon_x.view(batch_size,-1), x.view(batch_size, -1), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE, KLD

In [ ]:
from tqdm import tqdm

for ep_num in range(config.num_epochs): 
    model.train()
    train_mse, train_kld, train_loss = 0, 0, 0
    for index, image_attr in tqdm(enumerate(train_loader)):

        image = image_attr[0].to(config.device)

        optimizer.zero_grad()
        
        recon_batch, mu, logvar = model(image)
        
        mse_loss, kld_loss = loss_function(recon_batch, image, mu, logvar)
        loss = mse_loss + kld_loss
        loss.backward()
        optimizer.step()
        
        wandb.log({'train loss':loss.item() / len(image),
                   'MSE':mse_loss.item() / len(image),
                   'KL':kld_loss.item() / len(image)
                   })
        
        torch.nn.utils.clip_grad_norm_(model.encoder.parameters(), config.grad_clip_value)
        torch.nn.utils.clip_grad_norm_(model.decoder.parameters(), config.grad_clip_value)

        if (index + 1) % 500 == 0:
            fid = fid_VAE(config, fid_loader, model, classifier)
            wandb.log({'FID':fid})
            model.eval()

            for ind, image_attr in enumerate(val_loader):  # batch = 1
                if ind >= 10: break
                image = image_attr[0].to(config.device)

                fake_image, _, _ = model(image)

                image = image.detach().cpu()[0]
                fake_image = fake_image.detach().cpu()[0]

                wandb.log({"samples": [wandb.Image((image.permute(1, 2, 0).numpy() + 1) / 2, 
                                                    caption='real'),
                                       wandb.Image((fake_image.permute(1, 2, 0).numpy() + 1) / 2, 
                                                    caption='fake')]
                          })
            model.train()

    print('end of epoch', ep_num)
    torch.save(model.state_dict(), '/content/drive/MyDrive/dge3' + str(ep_num))
    scheduler.step()

In [ ]:
### generation of samples
model.eval()
samples = model.sample(config.batch_size)
for image in samples:
    wandb.log({"GENERATED": [wandb.Image((image.detach().cpu().permute(1, 2, 0).numpy() + 1) / 2, 
                                                    caption='generated from noise')]
              })

### Задача 2 (0.3 балла, но если не сделаете, за всю домашку max 0.1 за прошлый пункт):

После этого попробуем обучить обычный NF на Celeba до нормального качества, померить FID и запомнить для будущего сравнения

В качестве потока можно использовать все что вы хотите, Coupling/Autoregressive/Linear слои, любые трансформации. 

Можно использовать как и сверточные потоки, так и линейные (развернув селебу в один вектор)

### Real-NVP model

In [20]:
config = {
    'image_size': 64,
    'batch_size': 8,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'num_epochs': 30,
    'grad_clip_value': 1.5
}
config = DotDict(config)

### NF config
config['hid_size'] = 30

### Training and Optimization config
config['lr_start'] = 3e-4

wandb_start(config, 'NF-1')

In [21]:
from NF import AffineHalfFlow, NormalizingFlowModel
from torch.distributions import MultivariateNormal

# разворачиваю картинку в вектор!
vec_size = config.image_size * config.image_size * 3

In [22]:
### creating flows
flows = [AffineHalfFlow(dim=vec_size, 
                        parity=i%2, 
                        device=config.device) 
        for i in range(9)]

### creating prior
prior = MultivariateNormal(torch.zeros(vec_size).to(device=config.device), 
                           torch.eye(vec_size).to(device=config.device))

In [23]:
### creating model
model = NormalizingFlowModel(prior, flows, device=config.device)
wandb.watch(model)

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr_start)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.95, verbose=True)

Adjusting learning rate of group 0 to 1.0000e-03.


In [24]:
def evaluate(config, fid_loader, model, classifier, val_loader):
    fid = fid_NF(config, fid_loader, model, classifier)
    wandb.log({'FID':fid})

    # deprecation
    model.eval()

    for ind, image_attr in enumerate(val_loader):  # batch = 1
        if ind >= 10: break
        image = image_attr[0].view(1, -1).to(config.device)   # batch=1

        fake_image, _, _ = model(image)

        image = image.view(1, 3,
                          config.image_size,
                          config.image_size).detach().cpu()[0]
        fake_image = fake_image[-1].view(1, 3,
                                        config.image_size,
                                        config.image_size).detach().cpu()[0]


        # sample
        sampled_image = model.sample(config.batch_size)
        sampled_image = sampled_image[-1].view(config.batch_size, 3,
                                            config.image_size,
                                            config.image_size).detach().cpu()[0]
        
        wandb.log({"samples": 
                   [wandb.Image((image.permute(1, 2, 0).numpy() + 1) / 2, 
                                caption='real'),
                    wandb.Image((fake_image.permute(1, 2, 0).numpy() + 1) / 2, 
                                caption='fake')]
                  ,
                   "GENERATED":
                   [wandb.Image((sampled_image.permute(1, 2, 0).numpy() + 1) / 2, 
                    caption='generated from noise')]
                   })

    model.train()

In [ ]:
from tqdm import tqdm

for ep_num in range(config.num_epochs): 
    model.train()
    for index, image_attr in tqdm(enumerate(train_loader)):
        image = image_attr[0].view(config.batch_size, -1).to(config.device)

        # optimizer.zero_grad()
        model.zero_grad()

        zs, prior_logprob, log_det = model(image)
        logprob = prior_logprob + log_det
        loss = -torch.mean(logprob)   # NLL
        
        loss.backward()
        optimizer.step()

        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip_value)

        # tracking
        wandb.log({'NF train loss':loss.item() / vec_size})


    evaluate(config, fid_loader, model, classifier, val_loader)
    torch.save(model.state_dict(), '/content/drive/MyDrive/NF' + str(ep_num))
    #scheduler.step()

211it [02:13,  1.58it/s]

### Задача 3 (0.6 балла):

Попробуйте повторить архитектуру VAPNEV из https://arxiv.org/pdf/1611.05209.pdf. Сравните качество (FID) между тремя разными моделями

Здесь вы можете использовать VAE и NF из предыдущих пунктов, необходимо только понять как они совмещаются в оригинальной статье

В отчете напишите, почему по вашему мнению такой подход будет лучше (или может быть хуже) чем обычный VAE?



### Бонусная задача (0.2 балла):

Найдите, реализуйте и сравните с предыдущими моделями еще один интересный способ совмещения NF и VAE

##### Подсказки:

1. Если вы учите на колабе или на наших машинках, вероятнее всего что обучение будет очень долгим на картинках 256х256. Никто не мешает уменьшить разрешение, главное чтобы было видно что генерация выучились и качество было ок

2. Вы можете сделать ваш VAE/NF/VAPNEV условным, придумав как вы будете передавать в него conditional аттрибуты селебы

3. Не забывайте про аугментации


